<a href="https://colab.research.google.com/github/rameshhpathak/DisasterAssessment/blob/master/segmentation_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Basic_blocks = drive.CreateFile({'id':'1LIJahCBOgKswPjV3dCdjno6iQunL1JQN'})
#UNet = drive.CreateFile({'id':'1Xf0xSo-CswXiuiUR_tc9IbvzRx-UWU7M'})
UNet = drive.CreateFile({'id':'1OcRzMCigngoFCev3M_WrkS_oCg1ZqBgp'})
utils = drive.CreateFile({'id':'1VEQdnYI1HMy0mp8BtbdiGWz-BWSzDn5y'})

In [0]:
#Basic_blocks.GetContentFile('Basic_blocks.py')
UNet.GetContentFile('UNet_param.py')
utils.GetContentFile('utils.py')
#UNet.GetContentFile('UNet.py')

In [0]:
from UNet_param import *
import numpy as np


import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable

In [0]:
# hyperparameters
batch_size = 32
img_size = 256
lr = 0.00005
epoch = 10

In [0]:
img_dir = "/content/drive/My Drive/Major Project/dataset/building/crowdAI/train"

img_data = dset.ImageFolder(root=img_dir, transform=transforms.Compose([
    transforms.Resize(size=img_size),
    transforms.CenterCrop(size=(img_size, img_size * 2)),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True,pin_memory=True,num_workers=8)
print(img_data)

generator = UNet(pool_kernel=3, pool_stride=2,repeat_blocks=4, n_filters=16, out_channels=1).cuda()

# load pretrained model

path = "/content/drive/My Drive/Major Project/code/building/crowdAI/model/unet.pkl"

try:
    generator = torch.load(path)
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass
  
  
filepath = "/content/drive/My Drive/Major Project/code/building/crowdAI/error/mse2"

# loss function 

loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

# training

file = open(filepath, 'w')
for i in range(epoch):
    for _, (image, label) in enumerate(img_batch):
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)
        map_image = map_image[:,0:1,:,:]
        
        gen_optimizer.zero_grad()

        x = Variable(satel_image).cuda(0)
        y_ = Variable(map_image).cuda(0)
        y = generator.forward(x)

        loss = loss_func(y, y_)
        file.write(str(loss) + "\n")
        loss.backward()
        gen_optimizer.step()
           
            
        if _ % 500 == 0:
            torch.save(generator, path)
            print(i,loss)           
            #original = "/content/drive/My Drive/Major Project/code/building/crowdAI/result/filter_16/1/"+"original_image_{}_{}.png".format(i, _)
            #label = "/content/drive/My Drive/Major Project/code/building/crowdAI/result/filter_16/1/"+"label_image_{}_{}.png".format(i, _)
            #gen = "/content/drive/My Drive/Major Project/code/building/crowdAI/result/filter_16/1/"+"gen_image_{}_{}.png".format(i, _)
            #v_utils.save_image(x.cpu().data, original)
            #v_utils.save_image(y_.cpu().data, label)
            #v_utils.save_image(y.cpu().data, gen)
            

Dataset ImageFolder
    Number of datapoints: 256633
    Root Location: /content/drive/My Drive/Major Project/dataset/building/crowdAI/train
    Transforms (if any): Compose(
                             Resize(size=256, interpolation=PIL.Image.BILINEAR)
                             CenterCrop(size=(256, 512))
                             ToTensor()
                         )
    Target Transforms (if any): None


/content/UNet_param.py:22: UserWarning: Please make sure, that your input tensor's dimensions are divisible by (pool_stride ** repeat_blocks)
  warnings.warn("Please make sure, that your input tensor's dimensions are divisible by "



--------model restored--------

0 tensor(0.0192, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0231, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0172, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0253, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0193, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0196, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0219, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0173, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0184, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0183, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0204, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0212, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0170, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0157, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0244, device='cuda:0', grad_fn=<MseLossBackward>)
0 tensor(0.0183, device='cuda:0', grad_fn=<MseLossBa